# 训练的提速
实际训练后发现，速度没有预想得那么快，打开GPU性能窗口发现大多数时间GPU只是显存占用，算力占用为0,而CPU确实占满了的。
[GPU](../images/GPU.png)，经过测试和查阅资料，发现是因为CPU的读取速度过慢导致跟不上GPU的运算，进而使得大部分时间都是消耗在CPU调用数据，GPU都是闲置的，如果只有我的电脑这样倒也没啥，可能说明我的CPU比较垃圾，但是在kaggle上的测试也是这样[kaggle](../images/kaggle.png)，进一步查阅资料和测试发现，合理设置num_workers可以缓解这一问题，不同num_workers的耗时对比见下图[time_consume_w.png](../images/time_consume_w.png),可以看到，训练的时间基本都是稳定的，占比很小，占大头的都是进入和退出for循环和epoch，这个都是CPU读取数据的耗时，下面测试各种优化方法，看看哪种有效。

## 1.设置不同的num_workers

设置不同的num_workers确实可以影响，但是其他参数变了之后这个好像也要重新选，在batch_size = 32的情况下，测试不同batch_size的影响如下图所示：
在cornell上的测试结果：

[time_consume_c.png](../images/time_consume_c.png)

在jacquard上的测试结果：

[time_consume_j.png](../images/time_consume_j.png)

## 2.使用dali库
NVIDIA DALI（R）是NVIDIA的数据加载库，它是高度优化的构建基块和执行引擎的集合，可加速深度学习应用程序输入数据的预处理。DALI提供了性能和灵活性，可以作为一个库来加速不同的数据管道。然后可以将该库轻松集成到不同的深度学习训练和推理应用程序中。

主要就是想了利用它从磁盘批量读取图像文件并转换成Tensor文件，同时可以执行一定的增强操作。

下面按照官方的例程来试一下

In [1]:
from nvidia.dali.pipeline import Pipeline

help(Pipeline)

Help on class Pipeline in module nvidia.dali.pipeline:

class Pipeline(builtins.object)
 |  Pipeline class is the base of all DALI data pipelines. The pipeline
 |  encapsulates the data processing graph and the execution engine.
 |  
 |  Parameters
 |  ----------
 |  `batch_size` : int, optional, default = -1
 |      Batch size of the pipeline. Negative values for this parameter
 |      are invalid - the default value may only be used with
 |      serialized pipeline (the value stored in serialized pipeline
 |      is used instead).
 |  `num_threads` : int, optional, default = -1
 |      Number of CPU threads used by the pipeline.
 |      Negative values for this parameter are invalid - the default
 |      value may only be used with serialized pipeline (the value
 |      stored in serialized pipeline is used instead).
 |  `device_id` : int, optional, default = -1
 |      Id of GPU used by the pipeline.
 |      A None value for this parameter means that DALI should not use GPU nor CUDA

In [17]:
import os.path

test_data_root = os.environ['DALI_EXTRA_PATH']

# Caffe LMDB
lmdb_folder = os.path.join(test_data_root, 'db', 'lmdb')

N = 8             # number of GPUs
BATCH_SIZE = 128  # batch size per GPU
ITERATIONS = 32
IMAGE_SIZE = 3

In [18]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types

class CaffeReadPipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id, num_gpus):
        super(CaffeReadPipeline, self).__init__(batch_size, num_threads, device_id)

        self.input = ops.CaffeReader(path = lmdb_folder,
                                     random_shuffle = True, shard_id = device_id, num_shards = num_gpus)
        self.decode = ops.ImageDecoder(device = "mixed", output_type = types.RGB)
        self.resize = ops.Resize(device = "gpu",
                                 interp_type = types.INTERP_LINEAR)
        self.cmn = ops.CropMirrorNormalize(device = "gpu",
                                            dtype = types.FLOAT,
                                            crop = (227, 227),
                                            mean = [128., 128., 128.],
                                            std = [1., 1., 1.])
        self.uniform = ops.Uniform(range = (0.0, 1.0))
        self.resize_rng = ops.Uniform(range = (256, 480))

    def define_graph(self):
        inputs, labels = self.input(name="Reader")
        images = self.decode(inputs)
        images = self.resize(images, resize_shorter = self.resize_rng())
        output = self.cmn(images, crop_pos_x = self.uniform(),
                          crop_pos_y = self.uniform())
        return (output, labels)

In [19]:
import numpy as np
from nvidia.dali.plugin.pytorch import DALIGenericIterator

label_range = (0, 999)
pipes = [CaffeReadPipeline(batch_size=BATCH_SIZE, num_threads=2, device_id = device_id, num_gpus = N) for device_id in range(N)]
pipes[0].build()
dali_iter = DALIGenericIterator(pipes, ['data', 'label'], pipes[0].epoch_size("Reader"))
for i, data in enumerate(dali_iter):
    if i >= ITERATIONS:
        break
    # Testing correctness of labels
    for d in data:
        label = d["label"]
        image = d["data"]
        ## labels need to be integers
        assert(np.equal(np.mod(label, 1), 0).all())
        ## labels need to be in range pipe_name[2]
        assert((label >= label_range[0]).all())
        assert((label <= label_range[1]).all())
print("OK")

RuntimeError: Critical error when building pipeline:
Error when constructing operator: CaffeReader encountered:
[/opt/dali/dali/operators/reader/loader/lmdb.h:52] Assert on "mdb_env_open(mdb_env_, path.c_str(), mdb_flags, 0664) == 0" failed: LMDB Error: Invalid argument, with file: /opt/dali_extra/db/lmdb
Stacktrace (100 entries):
[frame 0]: /home/ldh/anaconda3/envs/gg_cnn/lib/python3.6/site-packages/nvidia/dali/libdali_operators.so(+0x3cf8ee) [0x7fa7aa7218ee]
[frame 1]: /home/ldh/anaconda3/envs/gg_cnn/lib/python3.6/site-packages/nvidia/dali/libdali_operators.so(+0x182a83b) [0x7fa7abb7c83b]
[frame 2]: /home/ldh/anaconda3/envs/gg_cnn/lib/python3.6/site-packages/nvidia/dali/libdali_operators.so(+0x1830cc9) [0x7fa7abb82cc9]
[frame 3]: /home/ldh/anaconda3/envs/gg_cnn/lib/python3.6/site-packages/nvidia/dali/libdali_operators.so(std::_Function_handler<std::unique_ptr<dali::OperatorBase, std::default_delete<dali::OperatorBase> > (dali::OpSpec const&), std::unique_ptr<dali::OperatorBase, std::